## Resources from XSEDE API

In [2]:
from sgci_resource import core
gh = core.SCGICatalogLocal(folder="./sgci-resource-inventory")


## List Resources

In [3]:
l = gh.listResources()
l


['resources-schema', 'stampede2.tacc.xsede', 'halstead.rcac.purdue']

## Load Stampede2 definition

In [4]:
stampede2 = gh.getResource('stampede2.tacc.xsede')
stampede2

{'schemaVersion': '1.0.0',
 'host': 'stampede2.tacc.xsede.org',
 'name': 'tacc-xsede-stampede2',
 'description': 'WIP: Sample resource for TACC Stampede2 Cluster',
 'computeResources': [{'schedulerType': 'BATCH',
   'connections': [{'connectionProtocol': 'SSH',
     'securityProtocol': 'SSHKEYS',
     'port': 22},
    {'connectionProtocol': 'SSH',
     'securityProtocol': 'PASSWORDS',
     'port': 22}],
   'batchSystem': {'jobManager': 'SLURM',
    'commandPaths': [{'name': 'SUBMISSION', 'path': '/bin/sbatch'}],
    'partitions': [{'name': 'normal',
      'totalNodes': 256,
      'nodeHardware': {'cpuType': 'KNL CPUs @ 1.40GHz',
       'cpuCount': 68,
       'memoryType': 'DDR4',
       'memorySize': '96 GB'}}],
    'executionCommands': [{'commandType': 'mpi',
      'commandPrefix': 'ibrun',
      'moduleDependencies': ['intel/17.0.4', 'impi/17.0.3']}]}},
  {'schedulerType': 'FORK',
   'connections': [{'connectionProtocol': 'SSH',
     'securityProtocol': 'SSHKEYS',
     'port': 22},
 

## get Host, name and Scheduler Types

In [5]:
gh.searchPath("[name, host, computeResources[].schedulerType]", stampede2)

['tacc-xsede-stampede2', 'stampede2.tacc.xsede.org', ['BATCH', 'FORK']]

## De-reference a template

In [8]:
instance_json = '''[{
    "instance" : {
        "name" : "GaAs_Tutorial",
        "execution" : {
            "host" : { "#ref" : "{res}#host" },
            "connection" : { "#ref" : "{res}#((computeResources[?schedulerType==\'BATCH\'] | [0]).connections[?securityProtocol==\'PASSWORDS\'] | [0])" },
            "execution" : { "#ref" : "{res}#(computeResources[?schedulerType==\'BATCH\'] | [0]).{jobManager:batchSystem.jobManager, commandPaths:batchSystem.commandPaths, partition:batchSystem.partitions[0]}"},
            "parallel" : { "#ref" : "{res}#((computeResources[?schedulerType==\'BATCH\'] | [0]).batchSystem.[executionCommands[?commandType==\'mpi\']][] | [0])" }
        },
        "storage":{
            "connection" : { "#ref" : "{res}#((computeResources[?schedulerType==\'BATCH\'] | [0]).connections[?securityProtocol==\'PASSWORDS\'] | [0])" },
            "filesystem" : { "#ref" : "{res}#(storageResources[?storageType==\'POSIX\'].fileSystems[] | [0])" }
        }
    }
}]'''
a1 = gh.derefResource(instance_json, {'res': stampede2})
a1

[{'instance': {'name': 'GaAs_Tutorial',
   'execution': {'host': 'stampede2.tacc.xsede.org',
    'connection': {'connectionProtocol': 'SSH',
     'securityProtocol': 'PASSWORDS',
     'port': 22},
    'execution': {'jobManager': 'SLURM',
     'commandPaths': [{'name': 'SUBMISSION', 'path': '/bin/sbatch'}],
     'partition': {'name': 'normal',
      'totalNodes': 256,
      'nodeHardware': {'cpuType': 'KNL CPUs @ 1.40GHz',
       'cpuCount': 68,
       'memoryType': 'DDR4',
       'memorySize': '96 GB'}}},
    'parallel': {'commandType': 'mpi',
     'commandPrefix': 'ibrun',
     'moduleDependencies': ['intel/17.0.4', 'impi/17.0.3']}},
   'storage': {'connection': {'connectionProtocol': 'SSH',
     'securityProtocol': 'PASSWORDS',
     'port': 22},
    'filesystem': {'homeDir': '/home1',
     'scratchDir': '/scratch',
     'workDir': '/work'}}}}]